In [14]:
import numpy as np
import pandas as pd

In [15]:

# Parameters
Length = 600
Width = 700
h_channel = 100
H_reservoir= [400 + i*5 for i in range(101)]

# Initialize arrays to store results
dhbyhar = np.zeros(len(H_reservoir))
Rcritar = np.zeros(len(H_reservoir))
Rar = np.zeros(len(H_reservoir))
dRar = np.zeros(len(H_reservoir))
Var = np.zeros(len(H_reservoir))
dVar = np.zeros(len(H_reservoir))

for i in range(len(H_reservoir)):
    l = Length  # length of inlet channel
    w = Width  # channel width in microns
    h = h_channel  # channel height in microns
    H = H_reservoir[i]  # step height in microns
    r = 0.5  # radius of curvature of thread
    Kth = 1 / r  # curvature of thread
    R = 0  # radius of curvature of the droplet
    Rstep = 0.01  # step by which to increase R in simulation, in microns
    Kb = 0  # curvature of the droplet
    rcrit = h / 2  # critical radius of thread after which it is unstable i.e. height of reservoir/2
    Kcrit = 1 / rcrit  # critical curvature 
    Rcrit = 0  # critical radius of droplet 
    X = 0  # aspect ratio of confined droplet
    fX = 0  # function of aspect ratio
    lcrit = np.pi * rcrit  # critical length of necking region (for Rayleigh instability)

    Rh = 0.001  # Average hydraulic resistance
    T = 0.005  # Surface tension between the inner and outer phase
    Qin = 2  # Inlet flowrate

    # Calculate Rcrit
    while True:
        R += Rstep
        X = 2 * R / H
        if 0 < X < 1:
            fX = 2 / X - 1
        elif X == 1:
            fX = 1
        elif X > 1:
            fX = np.pi / (4 * X)  # assuming X >> 1
        Kb = 2 * (1 + fX) / H
        if Kb <= Kth:
            break

    # Calculate Rcrit
    while True:
        R += Rstep
        X = 2 * R / H
        if 0 < X < 1:
            fX = 2 / X - 1
        elif X == 1:
            fX = 1
        elif X > 1:
            fX = np.pi / (4 * X)  # assuming X >> 1
        Kb = 2 * (1 + fX) / H
        Kth = Kb
        if Kb <= Kcrit:
            break

    Rcrit = R
    X = 2 * Rcrit / H

    # Calculate volumes
    dV1 = (r**2 * (np.pi - 4) + h**2 * (1 - np.pi / 4)) * l
    dV2 = np.pi * (w - h) * h**2 / 2
    if X > 1:
        V = np.pi * H * (Rcrit - H / 2) * ((Rcrit - H / 2) + np.pi * H / 4)
    else:
        V = 4 * np.pi * Rcrit**3 / 3

    V += dV1 + dV2

    if V > 4 * np.pi * H**3 / 24:
        R = (np.sqrt(np.pi**2 * H**2 / 16 + 4 * V / (np.pi * H)) - np.pi * H / 4) / 2 + H / 2
    else:
        R = (3 * V / (4 * np.pi))**(1 / 3)

    dV = dV1 + dV2
    dR = R - Rcrit
    dh = H - h

    # Store results
    dhbyhar[i] = dh / h
    Rcritar[i] = Rcrit
    Rar[i] = R
    dRar[i] = dR
    Var[i] = V
    dVar[i] = dV


In [16]:
# Combine results into a pandas DataFrame
df = pd.DataFrame({
    'lar': [Length] * len(H_reservoir),
    'war': [Width]* len(H_reservoir),
    'har': [h_channel] * len(H_reservoir),
    'Har': H_reservoir,
    'dhbyhar': dhbyhar,
    'Rcritar': Rcritar,
    'Rar': Rar,
    'Var': Var,
    'dVar': dVar
})

# Write DataFrame to CSV with labels
df.to_csv('results_sim.csv', index=False)